<img src="imgs/front.jpeg" width="1400">

<h1><center> Where to get the lecture notes </center></h1>

You can get copies of all the lecture files at my personal website. Go to:

http://www.astro.lu.se/~mikkola/

and click on **Teaching**


Each lecture contains (as notebooks)
- Manual 
- Exercises
- Presentation

---

<img src="imgs/numpy.png" width="300"/>

### What is NumPy?

From [numpy.org](https://numpy.org/devdocs/user/whatisnumpy.html)

> NumPy is the fundamental package for scientific computing in Python. 


NumPy is all about the *ndarray* object which is an *n*-dimensional array of homogenous data type. I.e. one single data type. Some differences between the NumPy array and Python sequences:

- NumPy arrays have fixed size at creation
- Allows advanced mathematical operations on large data

### *ndarray* example

Let's say we have two Python lists `a` and `b`. To multiply all the elements of the first array with the corresponding elements of the second we would do perhaps:


In [ ]:
a = [1, 2, 3]
b = [1, 10, 100]
c = []
for i in range(len(a)):
    c.append(a[i] * b[i])
    
print(c)

We can do the same thing a lot quicker in NumPy

In [ ]:
import numpy as np
a = np.array(a)
b = np.array(b)
c = a * b
print(c)

### The *ndarray*

Let's understand a few things about the ndarray. As specified, it can be $n$-dimensional

In [ ]:
import numpy as np
arr = np.array([[1, 2, 3], [4, 5, 6]])
arr

Check the array shape

In [ ]:
arr.shape

Get the number of dimensions

In [ ]:
arr.ndim

Total number of elements

In [ ]:
arr.size

### Creating arrays

We can create arrays from lists and tuples, mixes of lists/tuples, or from numpy itself.

In [ ]:
arr1 = np.array( [ [1, 2, 3], [4, 5, 6] ] )
arr2 = np.array( ( (1, 2, 3), (4, 5, 6) ) )
arr3 = np.array( [ (1, 2, 3), (4, 5, 6) ] )
arr4 = np.array( ( [1, 2, 3], [4, 5, 6] ) )
print(arr1, '\n')
print(arr2, '\n')
print(arr3, '\n')
print(arr4, '\n')

In [ ]:
arr1 = np.ones(3)
arr2 = np.zeros(3)
print(arr1, '\n')
print(arr2, '\n')

We can also specify the data type at creation

In [ ]:
arr1 = np.array([1, 2, 3], dtype=int)
arr2 = np.array([1, 2, 3], dtype=float)
arr3 = np.array([1, 2, 3], dtype=str)
arr4 = np.array([1, 2, 3], dtype=complex)
print(arr1, '\n')
print(arr2, '\n')
print(arr3, '\n')
print(arr4, '\n')

### Other things NumPy:
Let's quickly run through some NumPy basics

In [ ]:
# Sequences
print(np.arange(0, 100, 20))
print(np.linspace(0,100,5))
print(np.logspace(0,2,5))

In [ ]:
# Elementwise operations
A = np.array([[1, 1],[1, 1]])
B = np.array([[2, 2],[2, 2]])
print(A * B) # Elementwise
print(A @ B) # Matrix operation
print(A.dot(B)) # same matrix operation

In [ ]:
# ndarray methods
arr = np.arange(9).reshape(3,3)
print(arr)
print(arr.sum())
print(arr.min())
print(arr.max())

In [ ]:
# Along axis
print(arr.sum(axis=0))
print(arr.min(axis=1))
print(arr.max(axis=0))

In [ ]:
# maths
print(np.exp(2))
print(np.sqrt(9))
print(np.log(100))
print(np.log10(100))
print(np.add(2, 2))

### Indexing, slicing, and iterating
1-D arrays are indexed exactly like Python lists  

Multidimensional arrays have one index per axis and are accessed via tuple

In [ ]:
arr = np.arange(9).reshape(3,3) 
print(arr, '\n')  

print(arr[2,2]) 
print(arr[2,:])
print(arr[2])
print(arr[:,2])
print(arr[...,2]) 

<img src="imgs/indexing.png" width="1600" style="margin: -100px 0px 0px 100px;"/>

### Shape manipulation
Change shape

In [ ]:
arr = np.ones((2,6))
print(arr, '\n')

# ravel
print(arr.ravel(), '\n')

# reshape
print(arr.reshape(3,4), '\n')

# Transpose
print(arr.T, '\n')

Adding dimensions

In [ ]:
arr = np.ones(6)
print(arr.shape, '\n')

print(arr[:,np.newaxis].shape)
print(arr[np.newaxis,:].shape, '\n')

print(np.expand_dims(arr, axis=1).shape)
print(np.expand_dims(arr, axis=0).shape)

Stacking

In [ ]:
# 1D arrays
arr1 = np.ones(2)
arr2 = np.zeros(2)

print(np.row_stack((arr1, arr2)), '\n')
print(np.column_stack((arr1, arr2)))

In [ ]:
# 2D arrays
arr1 = np.ones((2,2))
arr2 = np.zeros((2,2))

print(np.vstack((arr1, arr2)), '\n')
print(np.hstack((arr1, arr2)), '\n')

print(np.concatenate((arr1,arr2),axis=0), '\n')
print(np.concatenate((arr1,arr2),axis=1))

### Broadcasting
Broadcasting is how NumPy treats arrays with different shapes when performing arithmetic operations. Typically it means that the smaller of the arrays is "stretched" out to match the larger's shape. For example:

<img src="imgs/np_multiply_broadcasting.png" width="1600" style="margin: -38px 0px 0px 100px;"/>

This can be rather useful for certain operations. For example consider the illustration below where we are simply doing the operation `arr_c = arr_a * arr_b`  
<br>
![](imgs/broadcasting.png)

The dimensions of two arrays are compatible with broadcasting when:
<br>
- They are equal
- One of them is 1

So let's see what works

```python
A      (3d array):  15 x 3 x 5
B      (3d array):  15 x 1 x 5
Result (3d array):  15 x 3 x 5
```

```python
A      (3d array):  15 x 3 x 5
B      (2d array):       3 x 5
Result (3d array):  15 x 3 x 5
```

```python
A      (3d array):  15 x 3 x 5
B      (2d array):       3 x 1
Result (3d array):  15 x 3 x 5
    
```

```python
A      (2d array):      2 x 1
B      (3d array):  8 x 4 x 3 # second from last dimensions mismatched
```

### Random numbers
Using the [`numpy.random`](https://numpy.org/devdocs/reference/random/index.html#numpyrandom) module it is possible to generate random numbers from a large number of different distributions.

In [ ]:
import numpy.random as random

# Random numbers between 0 and 1
print(random.rand(3), '\n')

# Random integers
print(random.randint(low=0, high=10, size=3), '\n')

# Random numbers sampled from uniform distribution
print(random.uniform(low=0, high=10, size=3), '\n')

# Random numbers sampled from Gaussian/Normal distribution
print(random.normal(size=3), '\n')

### [SciPy](https://www.scipy.org)

A collection of mathematocal algorithms and convenience functions built on NumPy.  

![](imgs/scipy_subpackages.png)

### [Astropy](https://www.astropy.org)

Python packages developed for astronomers.  

![](imgs/astropy.png)

# Now it's time to use the manual to solve the exercises. Good luck!